In [1]:
! pip install tokenizer sentencepiece
! nvidia-smi

You should consider upgrading via the '/usr/bin/python3.9 -m pip install --upgrade pip' command.
Wed Jan 18 17:12:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.76.02    Driver Version: 517.48       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 30%   33C    P8    26W / 350W |    796MiB / 24576MiB |     19%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+---------------

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import time
import torch._dynamo as torchdynamo
import torch
from kernl.model_optimization import optimize_model

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# default cache size needs to be increased to store the many graphs with generative models
torchdynamo.config.cache_size_limit = 512

model_name = "t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.eval().cuda()

tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [5]:
input_ids = tokenizer(
    "translate English to French: The house in the woods is wonderful, can we buy it ?",
    return_tensors="pt",
    padding='max_length', truncation=True, max_length=20
).to("cuda")

# vanilla

In [6]:
# vanilla
start = time.time()
for _ in range(10):
    output = model.generate(
        inputs=input_ids["input_ids"],
        min_length=22,
        max_length=22,
    )
    print(tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
end = time.time()
print(f'{(end - start) / 10} s / inference')

La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
0.42411463260650634 s / inference


In [7]:
# vanilla inference mode
with torch.inference_mode():
    start = time.time()
    for _ in range(10):
        output = model.generate(
            inputs=input_ids["input_ids"],
            min_length=22,
            max_length=22,
        )
        print(tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
    end = time.time()
    print(f'{(end - start) / 10} s / inference')

La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
0.33517866134643554 s / inference


# dynamo

In [8]:
# dynamo optimize
model.generate2 = torchdynamo.optimize("inductor")(model.generate)

/usr/local/lib/python3.9/dist-packages/torch/_dynamo/eval_frame.py:372: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled.Consider setting `torch.set_float32_matmul_precision('high')`
  warnings.warn(


In [9]:
# dynamo warm up
with torch.inference_mode():
    output = model.generate2(
        inputs=input_ids["input_ids"],
        min_length=22,
        max_length=22,
    )

Failed to collect metadata on function, produced code may be suboptimal.  Known situations this can occur are inference mode only compilation involving resize_ or prims (!schema.hasAnyAliasInfo() INTERNAL ASSERT FAILED); if your situation looks different please file a bug to PyTorch.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/torch/_functorch/aot_autograd.py", line 1273, in aot_wrapper_dedupe
    fw_metadata, _out, _num_aliasing_metadata_outs = run_functionalized_fw_and_collect_metadata(
  File "/usr/local/lib/python3.9/dist-packages/torch/_functorch/aot_autograd.py", line 289, in inner
    outs = f(*f_args)
  File "/usr/local/lib/python3.9/dist-packages/torch/_functorch/aot_autograd.py", line 2327, in functional_call
    out = Interpreter(mod).run(*args[params_len:], **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/torch/fx/interpreter.py", line 136, in run
    self.env[node] = self.run_node(node)
  File "/usr/local/lib/python3.9/dist-pac

In [12]:
# dynamo inference mode
with torch.inference_mode():
    start = time.time()
    for _ in range(10):
        output = model.generate2(
            inputs=input_ids["input_ids"],
            min_length=22,
            max_length=22,
        )
        print(tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
    end = time.time()
    print(f'{(end - start) / 10} s / inference')

La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
0.09746694564819336 s / inference


In [13]:
# dynamo inference mode on new input (Note: this does not trigger dynamo to recompile, and output is different and makes sense)

new_input_ids = tokenizer(
    "translate English to French: It is a nice day.",
    return_tensors="pt",
    padding='max_length', truncation=True, max_length=20
).to("cuda")

with torch.inference_mode():
    start = time.time()
    for _ in range(10):
        output = model.generate2(
            inputs=new_input_ids["input_ids"],
            min_length=22,
            max_length=22,
        )
        print(tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
    end = time.time()
    print(f'{(end - start) / 10} s / inference')

C'est une belle journée.........
C'est une belle journée.........
C'est une belle journée.........
C'est une belle journée.........
C'est une belle journée.........
C'est une belle journée.........
C'est une belle journée.........
C'est une belle journée.........
C'est une belle journée.........
C'est une belle journée.........
0.09948692321777344 s / inference


In [14]:
# dynamo inference mode on new input without output length constraint (Note: this does not trigger dynamo to recompile)

new_input_ids = tokenizer(
    "translate English to French: It is a nice day.",
    return_tensors="pt",
    padding='max_length', truncation=True, max_length=20
).to("cuda")

with torch.inference_mode():
    start = time.time()
    for _ in range(10):
        output = model.generate2(
            inputs=new_input_ids["input_ids"]
        )
        print(tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
    end = time.time()
    print(f'{(end - start) / 10} s / inference')

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


C'est une belle journée.
C'est une belle journée.
C'est une belle journée.
C'est une belle journée.
C'est une belle journée.
C'est une belle journée.
C'est une belle journée.


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


C'est une belle journée.
C'est une belle journée.
C'est une belle journée.
0.05882694721221924 s / inference


In [15]:
# dynamo inference mode on new input without output length constraint with sampling (Note: this does not trigger dynamo to recompile)

new_input_ids = tokenizer(
    "translate English to French: It is a nice day.",
    return_tensors="pt",
    padding='max_length', truncation=True, max_length=20
).to("cuda")

with torch.inference_mode():
    start = time.time()
    for _ in range(10):
        output = model.generate2(
            inputs=new_input_ids["input_ids"],
            do_sample=True
        )
        print(tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
    end = time.time()
    print(f'{(end - start) / 10} s / inference')

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
[2023-01-18 17:07:35,600] torch._inductor.ir: [WARNING] Using FallbackKernel: aten.topk
[2023-01-18 17:07:36,183] torch._inductor.ir: [WARNING] Using FallbackKernel: aten.topk
[2023-01-18 17:07:36,611] torch._inductor.ir: [WARNING] Using FallbackKernel: aten.topk
[2023-01-18 17:07:37,024] torch._inductor.ir: [WARNING] Using FallbackKernel: aten.topk
[2023-01-18 17:07:37,457] torch._inductor.ir: [WARNING] Using FallbackKernel: aten.topk
[2023-01-18 17:07:37,869] torch._inductor.ir: [WARNING] Using FallbackKernel: aten.topk
[2023-01-18 17:07:38,277] torch._in

C'est un beau jour.


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


C’est une journée agréable.
C'est une belle journée.
C’est une belle journée.
C’est un beau jour.
C'est une belle journée.


[2023-01-18 17:07:39,582] torch._inductor.ir: [WARNING] Using FallbackKernel: aten.topk
[2023-01-18 17:07:39,974] torch._inductor.ir: [WARNING] Using FallbackKernel: aten.topk


Il y a une belle journée.
C'est une belle journée.
C'est une belle journée.
Cette journée m'est agréable.
0.7632161617279053 s / inference


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [16]:
# dynamo inference mode on new input without output length constraint with beam-search (Note: this triggers dynamo to recompile, resulting in much longer infernce time due to dynamo warm-up)

new_input_ids = tokenizer(
    "translate English to French: It is a nice day.",
    return_tensors="pt",
    padding='max_length', truncation=True, max_length=20
).to("cuda")

with torch.inference_mode():
    start = time.time()
    for _ in range(10):
        output = model.generate2(
            inputs=new_input_ids["input_ids"],
            num_beams=3
        )
        print(tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
    end = time.time()
    print(f'{(end - start) / 10} s / inference')

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torch/nn/utils/stateless.py:44: UserWarning: functional_call was passed multiple values for tied weights. This behavior is deprecated and will be an error in future versions
  warnings.warn("functional_call was passed multiple values for tied weights. "
Failed to collect metadata on function, produced code may be suboptimal.  Known situations this can occur are inference mode only compilation involving resize_ or prims (!schema.hasAnyAliasInfo() INTERNAL ASSERT FAILED); if your situation looks different please file a b

C'est une belle journée.
C'est une belle journée.


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


C'est une belle journée.
C'est une belle journée.
C'est une belle journée.
C'est une belle journée.
C'est une belle journée.
C'est une belle journée.
C'est une belle journée.
C'est une belle journée.
13.003740215301514 s / inference


# kernl (Note: need to restart notebook and run since running dynamo first then run this would cause an error)

In [6]:
# kernl optimize
optimize_model(model.encoder)
optimize_model(model.decoder)

In [7]:
# kernl warmup with fp16
with torch.inference_mode(), torch.autocast(dtype=torch.float16, cache_enabled=True, device_type="cuda"):
    start = time.perf_counter()
    output = model.generate(inputs=input_ids["input_ids"], min_length=22, max_length=22)
    print(time.perf_counter() - start)
    print(tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))

143.9240474000003



In [9]:
# kernl inference model fp16
with torch.inference_mode(), torch.autocast(dtype=torch.float16, cache_enabled=True, device_type="cuda"):
    start = time.time()
    for _ in range(10):
        output = model.generate(
            inputs=input_ids["input_ids"],
            min_length=22,
            max_length=22,
        )
        print(tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
        print(output[0])
end = time.time()
print(f'{(end - start) / 10} s / inference')


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')


In [10]:
# kernl inference model fp16 on new input

new_input_ids = tokenizer(
    "translate English to French: It is a nice day.",
    return_tensors="pt",
    padding='max_length', truncation=True, max_length=20
).to("cuda")

with torch.inference_mode(), torch.autocast(dtype=torch.float16, cache_enabled=True, device_type="cuda"):
    start = time.time()
    for _ in range(10):
        output = model.generate(
            inputs=new_input_ids["input_ids"],
            min_length=22,
            max_length=22,
        )
        print(tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
        print(output[0])
end = time.time()
print(f'{(end - start) / 10} s / inference')


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')
